In [ ]:
#| default_exp basisblades

# Basis Blades

> Basis blades span the entire space.
>
> They are implemented using binary numbers.

In [ ]:
from dataclasses import dataclass, field
from collections import namedtuple

## What's a Basis?

Basis: A finite set $\mathcal{B} = \{\vec{b_1}, \vec{b_2}, ..., \vec{b_n}\}$ of vectors in a vector space $\vec{V}$ is a _basis_ for $\vec{V}$ if every vector in $\vec{V}$ is a linear combination of the vectors in $\mathcal{B}$ in one and only one way.

> Definition from [Linear and Geometric Algebra Chapter 2.5 Bases (MacDonald, 2011)](http://www.faculty.luther.edu/~macdonal/laga/)
>
> I highly recommend checking out [3Blue1Brown's explanation of Bases & Linear Combinations](https://youtu.be/k7RM-ot2NWY) for a more intuitive explanation

In Geometric Algebra, we have basis blades ($k$-dimensional vectors) which are denoted with $e_i$ instead of $\vec{b}_i$.

## Implementation


In 3D we have 8 basis $k$-blades: 

* 0-blades (scalars): $1$,
* 1-blades (vectors): $e_1, e_2, e_3$
* 2-blades (bivectors): $e_{12}, e_{13}, e_{23}$
* 3-blades (trivectors): $e_{123}$

Notice that every basis blade either contains a basis vector or it doesn't. For the 3D example, $e_{12}$ contains $e_1$ and $e_2$, but doesn't contain $e_3$.
In other words, we can encode every basis blade by the presence or absence of each basis blade. 
We use a bit to encode presence (1) or absence (0), and denote the basis blade by the position of each bit:

$$
{0 \atop e3} {0 \atop e2} {0 \atop e1}
$$

Therefore the whole space of 3D is encoded like this:

* scalar: $000$
* $e_1$: $001$
* $e_2$: $010$
* $e_3$: $100$
* $e_{12}$: $011$
* $e_{13}$: $101$
* $e_{23}$: $110$
* $e_{123}$: $111$

:::{.column-margin}
In general, we need $2^n$ bits to represent all basis blades of a $n$-dimensional space
:::

For simplicity, we'll stick to 2 dimensions for our impelementation. In 2D we have 4 basis $k$-blades: $1$ (unit scalar), $e1$, $e2$, and the unit bivector, $e_{12}$.

Computers store numbers in binary already, but Python defaults to printing numbers out in decimal. We can write binary numbers in Python as a string or integer, so we choose integer for convenience.

In [ ]:
# Binary string - must explicity convert to int
print('0b101', int('0b101', 2))
# Binary integer
print(0b101)

0b101 5
5


We could use a list to store all the binary numbers for each basis, but `namedtuple`s are more readable.

> Plus, when we import basis blades into other notebooks, we can import the tuple instead of each individual basis blade.

In [ ]:
GA2D = namedtuple('GA2D', ['scalar', 'e1', 'e2', 'e12'])
ga2d = GA2D(0b00, 0b01, 0b10, 0b11)
ga2d

GA2D(scalar=0, e1=1, e2=2, e12=3)

In [ ]:
# For convenience, we unpack the basis blades into the global scope
scalar, e1, e2, e12 = ga2d

### Weighted Basis Blades

While basis blades are defined with a weight of 1.0, we could associate a custom weight value to each basis blade using a class. This makes it easier to implement multivectors in later notebooks, since multivectors are a sum of weighted basis blades.

In our Python implementation, we group together the weight and basis data using a `dataclass`.

In [ ]:
@dataclass(frozen=True)
class BasisBlade:
    # --- Class variables (defined without type hint); shared by all BasisBlades
    bit_bases2d = {0b0: 's', 0b1: 'e1', 0b10: 'e2', 0b11: 'e12'}

    # --- Instance variables; made "immutable" by frozen=True
    # a.k.a scale or magnitude associated with blade
    weight: float = field(default=0.)
    # integer encoding 
    basis: int = field(default=0)
    # basis_name depends on basis, so it can only be set post-initialization
    basis_name: str = field(init=False)

    def __post_init__(self):
        # Update the "immutable" basis_name field
        object.__setattr__(self, 'basis_name', self.bit_bases2d.get(self.basis))

    def __str__(self) -> str:
        """Pretty print like math notation"""
        prettyprint = f'{self.weight if self.weight != 1 else ""}{self.basis_name}'
        return prettyprint

The `dataclass` decorator automagically implements useful methods for our `BasisBlade` class like `__eq__`, `__repr__`, `__init__`, and more. See this [mCoding video](https://youtu.be/vBH6GRJ1REM) for an overview, and the [dataclasses docs](https://docs.python.org/3/library/dataclasses.html) for details.

Also note that `bit_bases2d` is a _class variable_ which means every instance of this class shares the same `bit_bases2d`. Contrast this with the _instance variables_ `weight`, `basis`, and `basis_name`.

Finally, note that we use `object.__set_attr__(...)` to modify the `BasisBlade.basis_name` attribute, despite the class being "frozen". This is the only time a `BasisBlade` attribute needs to change post-initialization. See [this StackOverflow post](https://stackoverflow.com/questions/53756788/how-to-set-the-value-of-dataclass-field-in-post-init-when-frozen-true) for more details.

In [ ]:
# example basis blade
BasisBlade(4.0, e1)

BasisBlade(weight=4.0, basis=1, basis_name='e1')

The `BasisBlade` class supports pretty printing like math notation

In [ ]:
# Weight and basis
print(BasisBlade(4.0, e1))
# If weight=1. than only basis shown
print(BasisBlade(1.0, e12))
# 's' (for scalar) helps distinguish BasisBlade class instance from Python floats
print(BasisBlade(-12., scalar))

4.0e1
e12
-12.0s


In [ ]:
# dataclass automagically implements __eq__ method for us
BasisBlade(-3.4, e2) == BasisBlade(-3.4, e2)

True

Besides `__eq__`, no other operators are overloaded! This keeps the `BasisBlade` class small, and makes the implementation of operators simpler to understand.

> Of course, lack of operator overloading makes our implementation more inconvenient to use. For a fully-featured, operator overloaded Geometric Algebra library see [Clifford](https://github.com/pygae/clifford)

In [ ]:
# We cannot multiply floats and BasisBlades using '*' without operator overloading!
try:
    3.4 * BasisBlade(1., e1)
except TypeError as err:
    print(err)

unsupported operand type(s) for *: 'float' and 'BasisBlade'


## Readings

Chapter 2.3 Basis Blades

## What's Next?

Next, we implement multivectors.

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()